In [18]:
import numpy as np
import subprocess
from PIL import Image
import pyqrcode
from pyzbar.pyzbar import decode
import os

In [24]:
def get_max_value(version, errorL):
    numeric ={'1': {'L': 41,'M': 34, 'Q': 27, 'H': 17}, '2': {'L': 77,'M': 63, 'Q': 48, 'H': 34}, '3': {'L': 127,'M': 101, 'Q': 77, 'H': 58}, '4': {'L': 187,'M': 149, 'Q': 111, 'H': 82}, '5': {'L': 255,'M': 202, 'Q': 144, 'H': 106}, '6': {'L': 322,'M': 255, 'Q': 178, 'H': 139}, '7': {'L': 370,'M': 293, 'Q': 207, 'H': 154}, '8': {'L': 461,'M': 365, 'Q': 259, 'H': 202}, '9': {'L': 552,'M': 432, 'Q': 312, 'H': 235}, '10': {'L': 652,'M': 513, 'Q': 364, 'H': 288}, '11': {'L': 772,'M': 604, 'Q': 427, 'H': 331}, '12': {'L': 883,'M': 691, 'Q': 489, 'H': 374}, '13': {'L': 1022,'M': 796, 'Q': 580, 'H': 427}, '14': {'L': 1101,'M': 871, 'Q': 621, 'H': 468}, '15': {'L': 1250,'M': 991, 'Q': 703, 'H': 530}, '16': {'L': 1408,'M': 1082, 'Q': 775, 'H': 602}, '17': {'L': 1548,'M': 1212, 'Q': 876, 'H': 674}, '18': {'L': 1725,'M': 1346, 'Q': 948, 'H': 746}, '19': {'L': 1903,'M': 1500, 'Q': 1063, 'H': 813}, '20': {'L': 2061,'M': 1600, 'Q': 1159, 'H': 919}, '21': {'L': 2232,'M': 1708, 'Q': 1224, 'H': 969}, '22': {'L': 2409,'M': 1872, 'Q': 1358, 'H': 1056}, '23': {'L': 2620,'M': 2059, 'Q': 1468, 'H': 1108}, '24': {'L': 2812,'M': 2188, 'Q': 1588, 'H': 1228}, '25': {'L': 3057,'M': 2395, 'Q': 1718, 'H': 1286}, '26': {'L': 3283,'M': 2544, 'Q': 1804, 'H': 1425}, '27': {'L': 3517,'M': 2701, 'Q': 1933, 'H': 1501}, '28': {'L': 3669,'M': 2857, 'Q': 2085, 'H': 1581}, '29': {'L': 3909,'M': 3035, 'Q': 2181, 'H': 1677}, '30': {'L': 4158,'M': 3289, 'Q': 2358, 'H': 1782}, '31': {'L': 4417,'M': 3486, 'Q': 2473, 'H': 1897}, '32': {'L': 4686,'M': 3693, 'Q': 2670, 'H': 2022}, '33': {'L': 4965,'M': 3909, 'Q': 2805, 'H': 2157}, '34': {'L': 5253,'M': 4134, 'Q': 2949, 'H': 2301}, '35': {'L': 5529,'M': 4343, 'Q': 3081, 'H': 2361}, '36': {'L': 5836,'M': 4588, 'Q': 3244, 'H': 2524}, '37': {'L': 6153,'M': 4775, 'Q': 3417, 'H': 2625}, '38': {'L': 6479,'M': 5039, 'Q': 3599, 'H': 2735}, '39': {'L': 6743,'M': 5313, 'Q': 3791, 'H': 2927}, '40': {'L': 7089,'M': 5596, 'Q': 3993, 'H': 3057}}
    m = ''.join(str(x) for x in np.random.randint(0, 2, (numeric[version][errorL],)).tolist())
    return m

In [25]:
print (get_max_value('1', 'L'))

01111010111000011011000010101001000001101


In [11]:
def get_qr_array(m):
    qr = pyqrcode.create(m, error='H')
    t = np.array([[1-int(z) for z in x] for x in qr.text().split('\n')[:-1]])
    return t

In [6]:
def qr_rgb_1_encode(frame, size=(400, 400)):
    m = ''.join(str(x) for x in np.random.randint(0, 2, (10,)).tolist())
    t = get_qr_array(m)
    rgb = np.repeat(t[:, :, np.newaxis], 3, axis=2)
    rgb = np.uint8(rgb * 255)
    img = Image.fromarray(rgb, 'RGB')
    img = img.resize(size)
    frame.paste(img)
    return frame, m

In [4]:
def qr_rgb_3_encode(frame, size=(400, 400)):
    ms = []
    rgb = []
    for i in range(3):
        m = ''.join(str(x) for x in np.random.randint(0, 2, (100,)).tolist())
        ms.append(m)
        rgb.append(get_qr_array(m))
    rgb = np.stack(rgb, axis=2)
    rgb = np.uint8(rgb * 255)
    img = Image.fromarray(rgb, 'RGB')
    img = img.resize(size)
    frame.paste(img)
    return frame, ms

In [5]:
def qr_rgb_1_decode(frame, size=(400, 400)):
    f = frame.crop((0, 0, size[0], size[1]))
    res = decode(f)
    if not res:
        return ''
    return decode(f)[0].data.decode('utf-8')
   

In [6]:
def qr_rgb_3_decode(frame, size=(400, 400)):
    f = np.array(frame.crop((0, 0, size[0], size[1])))
    ms = []
    for i in range(3):
        t = f[:, :, i][:, :, np.newaxis]
        t = np.repeat(t, 3, axis=2)
        t = Image.fromarray(t, 'RGB')
        res = decode(t)
        if res:
            ms.append(res[0].data.decode('utf-8'))
        else:
            ms.append([])
    return ms

In [7]:
a =  [1,3]
a.append([1,2])
print a

[1, 3, [1, 2]]


In [8]:
def put_data(directory, version=0):
    '''embedes all of the png files in a directory with a qr code and saves them in place.
    returns the string representations of the qr codes in a list
    version 0 stands for 1 channel version 1 stands for 2 channel'''
    if version == 0:
        m = len(os.listdir(directory)) * [None]
    if version == 1:
        m = []
    for i, img in enumerate(os.listdir(directory)):
        frame = Image.open(directory + '/' + img)
        if version == 0:
            f, m[i] = qr_rgb_1_encode(frame)
        elif version == 1:
            f, k = qr_rgb_3_encode(frame)
            m.append(k[0])
            m.append(k[1])
            m.append(k[2])
        f.save(directory + '/' + img)
        if not i % 10:
            print i
    return m

In [9]:
def get_data(directory, version=0):
    '''reads the qr codes in all of the png files in the directory
    returns the string representations of the qr codes in a list
    version 0 stands for 1 channel version 1 stands for 2 channel'''
    if version == 0:
        m = len(os.listdir(directory)) * [None]
    if version == 1:
        m = []
    for i, img in enumerate(os.listdir(directory)):
        frame = Image.open(directory + '/' + img)
        if version == 0:
            m[i] = qr_rgb_1_decode(frame)
        elif version == 1:
            k = qr_rgb_3_decode(frame)
            m.append(k[0])
            m.append(k[1])
            m.append(k[2])

    return m
    
    

In [10]:
print 'start'
init_m = put_data('frames',1)
print 'qr codes embedded'
subprocess.call(["ffmpeg", "-i", "frames/image-%04d.png", "-c:v", "libvpx", "temp.webm"])
print "done encoding"
subprocess.call(["ffmpeg", "-i", "temp.webm", "-vf", "scale=800:600", "outframes/image-%04d.png"])
print "done decoding"
final_m = get_data('outframes',1)

ctr = 0
for i, f in zip(init_m, final_m):
    if i == f:
        ctr += 1
print str(ctr * 1.0 /len(init_m) * 100) + "% of the qr codes were decoded"
print ctr

subprocess.call(["rm", "temp.webm"])
subprocess.call(["rm", "outframes/*"])



#frame = Image.open('frames/image-0422.png')
#f, m0 = qr_rgb_3_encode(frame)
#m1 = qr_rgb_3_decode(f)
#print(m0 == m1)

start
0
10
20
30
40
50
60
70
80
90
100
110
120
qr codes embedded
done encoding
done decoding
40.2203856749% of the qr codes were decoded
146


1

In [11]:
frame = Image.open('frames/image-0422.png')
# f = np.array(frame.convert('HSV'))
f = np.array(frame.convert('YCbCr'))
print(np.max(f[:, :, 0]), np.min(f[:, :, 0]))
f[:, :, 0] = np.clip(f[:, :, 0] + 5, 0, 255)
print(np.max(f[:, :, 0]), np.min(f[:, :, 0]))
# img = Image.fromarray(f, 'HSV')
img = Image.fromarray(f, 'YCbCr')
img.convert('RGB')

IOError: [Errno 2] No such file or directory: 'frames/image-0422.png'